# Kristdemokraternas budgetmotion 2022

https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/hela-sverige-ska-fungera--kristdemokraternas_H9024221

In [5]:
import pandas as pd
import requests

In [6]:
# currency multiplier used in this budget
multiplier = 1_000

In [115]:
docs = [
    {'utgiftsområde': 1, 'dok_id': 'H9024186'},
    {'utgiftsområde': 2, 'dok_id': 'H9024200'},
    {'utgiftsområde': 3, 'dok_id': 'H9024202'},
    {'utgiftsområde': 4, 'dok_id': 'H9024192'},
    {'utgiftsområde': 5, 'dok_id': 'H9023883'},
    {'utgiftsområde': 6, 'dok_id': 'H9023908'},
    {'utgiftsområde': 7, 'dok_id': 'H9023882'},
    {'utgiftsområde': 8, 'dok_id': 'H9024214'},
    {'utgiftsområde': 9, 'dok_id': 'H9024172'},
    {'utgiftsområde': 10, 'dok_id': 'H9024213'},
    {'utgiftsområde': 11, 'dok_id': 'H9024215'},
    {'utgiftsområde': 12, 'dok_id': 'H9024166'},
    {'utgiftsområde': 13, 'dok_id': 'H9024169'},
    {'utgiftsområde': 14, 'dok_id': 'H9024198'},
    {'utgiftsområde': 15, 'dok_id': 'H9024160'},
    {'utgiftsområde': 16, 'dok_id': 'H9024163'},
    {'utgiftsområde': 17, 'dok_id': 'H9024162'},
    {'utgiftsområde': 18, 'dok_id': 'H9024222'},
    {'utgiftsområde': 19, 'dok_id': 'H9024219'},
    {'utgiftsområde': 20, 'dok_id': 'H9023881'},
    {'utgiftsområde': 21, 'dok_id': 'H9024195'},
    {'utgiftsområde': 22, 'dok_id': 'H9023879'},
    {'utgiftsområde': 23, 'dok_id': 'H9023880'},
    {'utgiftsområde': 24, 'dok_id': 'H9024217'},
    {'utgiftsområde': 25, 'dok_id': 'H9024201'}
]

In [116]:
def find_matching_table(tables):
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    for table in tables:
        if table.columns.shape == (4,) and 'Avvikelse från regeringen' in table.columns:
            break
    return table

In [124]:
def fetch_table(url, area):
    tables = pd.read_html(url, encoding='utf8', header=0)
    cols = ['Anslag', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    df = find_matching_table(tables)
    df.columns = cols
    df = df.dropna(how='all')
    df = df[~df.Anslag.str.startswith('Summa')]
    df['Utgiftsområde'] = area
    return df

In [125]:
tables = []

for doc in docs:
    try:
        url = f'http://data.riksdagen.se/dokument/{doc["dok_id"]}.html'
        table = fetch_table(url, area=doc['utgiftsområde'])
        tables.append(table)
    except ValueError:  # missing images...
        print(doc['utgiftsområde'])

13


In [126]:
df = pd.concat(tables, sort=False)

In [127]:
df = df.dropna(how='all')

In [128]:
df = df.drop('Regeringens förslag', axis=1)

In [129]:
df = df.replace('±0', '0')
df = df.rename(columns={'Avvikelse från regeringen': '2022'})
df['2022'] = df['2022'].str.replace('\s+', '', regex=True)
df['2022'] = df['2022'].str.replace('−', '-')

In [130]:
df['2022'] = df['2022'].astype(int) * multiplier

In [137]:
df

,Anslag,Namn,2022,Utgiftsområde
0,1:1,Kungliga hov- och slottsstaten,0,1
1,2:1,Riksdagens ledamöter och partier m.m.,0,1
2,2:2,Riksdagens förvaltningsanslag,0,1
3,2:3,Riksdagens fastighetsanslag,0,1
4,2:4,Riksdagens ombudsmän (JO),0,1
...,...,...,...,...
0,1:1,Kommunalekonomisk utjämning,4824000000,25
1,1:2,Utjämningsbidrag för LSS-kostnader,0,25
2,1:3,Bidrag till kommunalekonomiska organisationer,0,25
3,1:4,Tillfälligt stöd till enskilda kommuner och re...,0,25


In [139]:
# Manual addition of utgiftsområde 13 (from PDF)
df = df.append([
    {
        'Anslag': '1:1',
        'Namn': 'Etableringsåtgärder',
        '2022': 120_000_000,
        'Utgiftsområde': 13
    },
    {
        'Anslag': '3:1',
        'Namn': 'Särskilda jämställdhetsåtgärder',
        '2022': 27_000_000,
        'Utgiftsområde': 13
    },
    {
        'Anslag': '3:2',
        'Namn': 'Jämställdhetsmyndigheten',
        '2022': -26_000_000,
        'Utgiftsområde': 13
    },
    {
        'Anslag': '4:1',
        'Namn': 'Åtgärder mot segregation',
        '2022': -250_000_000,
        'Utgiftsområde': 13
    },
    {
        'Anslag': '4:2',
        'Namn': 'Delegationen mot segregation',
        '2022': -9_000_000,
        'Utgiftsområde': 13
    },
    {
        'Anslag': '99:2',
        'Namn': 'Inrättande av jämställdhetsdelegation',
        '2022': 13_000_000,
        'Utgiftsområde': 13
    },
])

df = df.sort_values(['Utgiftsområde', 'Anslag'])

In [141]:
df.to_csv('../data/budgetmotion-2022-kd.csv', index=False)